# Goals

* Check on the test and/or production databases

In [12]:
from dotenv import load_dotenv
load_dotenv()

True

In [13]:
import os
import pandas as pd
from pypika import Query, Table, Field, Column, Criterion

In [14]:
from SRAgent.db.connect import db_connect
from SRAgent.db.upsert import db_upsert
from SRAgent.db.utils import db_list_tables, db_glimpse_tables, db_get_table, execute_query
from SRAgent.db.get import db_find_srx
from SRAgent.db.create import create_table, create_table_router

In [15]:
# list all of the tables in prod
os.environ['DYNACONF'] = 'prod'
conn_prod = db_connect() 
print("\n".join(db_list_tables(conn_prod)))

eval
screcounter_log
srx_srr
srx_metadata
screcounter_star_params
screcounter_star_results


# Production

In [5]:
db_list_tables(conn_prod)

['eval',
 'screcounter_log',
 'srx_srr',
 'srx_metadata',
 'screcounter_star_params',
 'screcounter_star_results']

In [6]:
db_glimpse_tables(conn_prod)

#-- Table: eval --#
dataset_id	database	entrez_id	srx_accession	is_illumina	is_single_cell	is_paired_end	lib_prep	tech_10x	organism	cell_prep	created_at	updated_at
eval1	sra	29115023	ERX11157721	yes	yes	yes	10x_Genomics	3_prime_gex	zebrafish	single_cell	2025-01-03 18:38:01.789148	2025-01-03 18:38:01.789148
eval1	sra	36178506	ERX11887200	yes	yes	yes	10x_Genomics	3_prime_gex	mouse	single_cell	2025-01-03 18:38:01.789148	2025-01-03 18:38:01.789148
eval1	sra	5920965	ERX2188110	yes	yes	yes	other	not_applicable	macaque	single_cell	2025-01-03 18:38:01.789148	2025-01-03 18:38:01.789148
eval1	sra	34439895	ERX9692805	yes	yes	yes	10x_Genomics	3_prime_gex	chicken	single_cell	2025-01-03 18:38:01.789148	2025-01-03 18:38:01.789148
eval1	gds	200271457	NaN	no	no	no	not_applicable	not_applicable	human	not_applicable	2025-01-03 18:38:01.789148	2025-01-03 18:38:01.789148

#-- Table: screcounter_log --#
sample	accession	process	step	status	message	created_at	updated_at
ERX11662340	ERR12251984	fq-dump	check_

In [7]:
# read in srx-metadata as pandas dataframe
tbl = Table("srx_metadata")
stmt = Query \
    .from_(tbl) \
    .select("*") \
    .distinct() \
    .where(tbl.notes != "Processed by Chris Carpenter")
srx_metadata = pd.read_sql(str(stmt), conn_prod)
srx_metadata

,database,entrez_id,srx_accession,is_illumina,is_single_cell,is_paired_end,lib_prep,tech_10x,cell_prep,organism,tissue,disease,purturbation,cell_line,czi_collection_id,czi_collection_name,notes,created_at,updated_at
0,sra,26659536,SRX19392127,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,mouse,S1 cortex,neuropsychiatric outcomes,maternal immune activation,"microglia, P14, male, C57Bl/6N, WT",None,None,Metadata obtained by SRAgent,2025-01-06 22:15:53.177758,2025-01-06 22:19:05.896057
1,sra,27454961,ERX10668462,yes,yes,yes,10x_Genomics,other,single_cell,mouse,ILC progenitors from femur,unsure,unsure,unsure,None,None,Metadata obtained by SRAgent,2025-01-06 22:15:53.177758,2025-01-06 22:19:02.038324
2,sra,27454971,ERX10668472,yes,yes,yes,other,not_applicable,single_cell,mouse,ILC progenitors from femur,unsure,unsure,ILC2 cells,None,None,Metadata obtained by SRAgent,2025-01-06 22:15:53.177758,2025-01-06 22:38:48.116573
3,sra,27455317,ERX10668818,yes,yes,yes,other,not_applicable,single_cell,mouse,femur,unsure,Bcl11b regulatory networks,unsure,None,None,Metadata obtained by SRAgent,2025-01-07 00:28:36.611049,2025-01-07 00:30:22.045996
4,sra,27694563,SRX20273630,yes,yes,yes,other,not_applicable,single_cell,human,HCT116,not specified,infected,HCT116,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:44:17.048627
5,sra,29919592,SRX22006379,yes,yes,yes,Smart-seq2,not_applicable,single_cell,human,blastocyst,unsure,unsure,inner cell mass (ICM),None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:43:57.602382
6,sra,30864469,SRX22822524,yes,yes,yes,MARS-seq,not_applicable,single_cell,mouse,tumor,unsure,unsure,OT1 CD8+,None,None,Metadata obtained by SRAgent,2025-01-06 22:15:53.177758,2025-01-06 22:18:04.534748
7,sra,30865352,SRX22823407,yes,yes,yes,other,not_applicable,single_cell,mouse,tumor,phaeohyphomycosis,anti-PD1 treatment,endogenous CD8+ T cells,None,None,Metadata obtained by SRAgent,2025-01-03 18:42:38.083215,2025-01-03 18:45:48.003195
8,sra,30865785,SRX22823840,yes,yes,yes,MARS-seq,not_applicable,single_cell,mouse,tumor,cancer immunotherapy,none specified,"B16-OVA, MC38, LLC",None,None,Metadata obtained by SRAgent,2025-01-06 22:15:53.177758,2025-01-06 22:18:33.867744
9,sra,32469529,SRX24158623,yes,yes,yes,10x_Genomics,3_prime_gex,single_cell,human,primary cell culture,glaucoma,primary hTM cells isolated from human donor co...,healthy,None,None,Metadata obtained by SRAgent,2025-01-06 22:37:10.538238,2025-01-06 22:40:04.684759


# --OLD--

### Filter out records with no SRX accessions

`New dataset found by Find-Datasets agent`

In [8]:
entrez_ids = srx_metadata[srx_metadata["srx_accession"].isna()]["entrez_id"].unique().tolist()
entrez_ids

[]

In [13]:
# delete extrez_ids from srx_metadata
tbl = Table("srx_metadata")
stmt = Query \
    .from_(tbl) \
    .delete() \
    .where(tbl.entrez_id.isin(entrez_ids))
with conn_prod.cursor() as cur:
    cur.execute(str(stmt))
    conn_prod.commit()

In [16]:
# delete from log
tbl = Table("screcounter_log")
to_rm = ["ERX11146221"]
stmt = Query \
    .from_(tbl) \
    .delete() \
    .where(tbl.sample.isin(to_rm))
with conn_prod.cursor() as cur:
    cur.execute(str(stmt))
    conn_prod.commit()